Most of this code was adapted from:

Ree, N.; Göller, A. H.; Jensen, J. H. RegioML: Predicting the Regioselectivity of Electrophilic Aromatic Substitution Reactions Using Machine Learning. Digital Discovery 2022. https://doi.org/10.1039/D1DD00032B.

In [21]:
## Local for data files - update with latest Reaxys data
local_data = "/Users/hmm/PhD/Reaxys_Datasets/2022-05-06"
%cd {local_data}
!pwd

/Users/hmm/PhD/Reaxys_Datasets/2022-05-06
/Users/hmm/PhD/Reaxys_Datasets/2022-05-06


In [22]:
dependencies = "/Users/hmm/PhD/JupyterNotebooks/Dependencies/data_cleaning/"
%cd {dependencies}

/Users/hmm/PhD/JupyterNotebooks/Dependencies/data_cleaning


In [23]:
## Notebook working directory
work_dir = "/Users/hmm/PhD/JupyterNotebooks"
%cd {work_dir}

/Users/hmm/PhD/JupyterNotebooks


# Import Modules

In [24]:
import numpy as np
import pandas as pd

from functools import reduce
import operator
import copy
from tqdm import tqdm
from natsort import natsorted, index_natsorted

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdFMCS
from rdkit.Chem import rdmolops
from rdkit.Chem.Draw import rdMolDraw2D
"""
## To use widgets on data exploration

import ipywidgets as widgets
from ipywidgets import interact,fixed
from IPython.display import SVG,Image
from IPython.core.display import display
"""

%cd {dependencies}
from reorder_atoms import single_bonded_mol, reorder_labels, get_atoms_in_order
from locate_EAS_sites import locate_sites
from find_atoms import find_identical_atoms
%cd {work_dir}

/Users/hmm/PhD/JupyterNotebooks/Dependencies/data_cleaning
/Users/hmm/PhD/JupyterNotebooks


# Main Functions

In [25]:
def mol_with_atom_index(rdkit_mol):
    m = copy.deepcopy(rdkit_mol) 
    for atom in m.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx())
    return m


def modifify_mol(rdkit_smi):
    """ correct nitrogen charges  """

    rdkit_mol = Chem.MolFromSmiles(rdkit_smi, sanitize=False)
    rdkit_mol.UpdatePropertyCache(strict=False)
    rdmolops.SetAromaticity(rdkit_mol)

    rxn_smarts = ['[*;R:1]-[#7;R:2](=[*;R:3])=[#7;!R:4]-[*:5]>>[*;R:1]-[#7;R+:2](=[*;R:3])-[#7;!R-:4]-[*:5]',
                  '[#8;!R:1]=[#6;R:2]1-[#6;R:3]=[#7;R:4](-[*:5])=[#7;R:6]-[#8;R:7]1>>[#8;!R-:1]-[#6;R:2]-1=[#6;R:3]-[#7;R+:4](-[*:5])=[#7;R:6]-[#8;R:7]-1',
                  '[#6;R:1]=[#7;R:2]=[#7;R:3]-[#6;R:4]=[#7;R:5]>>[#6;R:1]=[#7;R+:2]-[#7;R-:3]-[#6;R:4]=[#7;R:5]',
                  '[*:1]-[#6;!R:2](=[O;!R:3])-[#7;!R:4]=[#6;R:5]1-[#8;R:6]-[#7;R:7]=[#7;R:8](-[*:9])=[#6;R:10]1>>[*:1]-[#6;!R:2](=[O;!R:3])-[#7;!R-:4]-[#6;R:5]=1-[#8;R:6]-[#7;R:7]=[#7;R+:8](-[*:9])-[#6;R:10]=1']

    fragments = Chem.GetMolFrags(rdkit_mol,asMols=True,sanitizeFrags=False)

    for i, fragment in enumerate(fragments):
        for smarts in rxn_smarts:
            patt = Chem.MolFromSmarts(smarts.split(">>")[0])
            while fragment.HasSubstructMatch(patt):
                rxn = AllChem.ReactionFromSmarts(smarts)
                ps = rxn.RunReactants((fragment,))
                fragment = ps[0][0]
                Chem.SanitizeMol(fragment)
        if i == 0:
            rdkit_mol = fragment
        else:
            rdkit_mol = Chem.CombineMols(rdkit_mol, fragment)
    
    rdmolops.Kekulize(rdkit_mol,clearAromaticFlags=True)
    rdkit_mol = Chem.MolFromSmiles(Chem.MolToSmiles(rdkit_mol))
    return rdkit_mol

In [26]:
def get_unique_reps2(rdkit_smis):
    unique_mols = []
    unique_smiles = []
    for rdkit_smi in rdkit_smis:
        try:
            rdkit_mol = modifify_mol(rdkit_smi)
            Chem.SanitizeMol(rdkit_mol)
            rdkit_smi = Chem.MolToSmiles(rdkit_mol, isomericSmiles=True)
        except:
            continue
        if rdkit_smi not in unique_smiles:
            unique_smiles.append(rdkit_smi)
            unique_mols.append(Chem.MolFromSmiles(rdkit_smi, sanitize=False))
    return unique_mols, unique_smiles

In [27]:
def get_reps2(rdkit_smis):
    mols = []
    smiles = []
    for rdkit_smi in rdkit_smis:
        try:
            rdkit_mol = modifify_mol(rdkit_smi)
            Chem.SanitizeMol(rdkit_mol)
            rdkit_smi = Chem.MolToSmiles(rdkit_mol, isomericSmiles=True)
        except:
            continue
        smiles.append(rdkit_smi)
        mols.append(Chem.MolFromSmiles(rdkit_smi, sanitize=False))
    return mols, smiles

In [28]:
def get_mols(rdkit_smis):
    mols = []
    unique_smiles = []
    for rdkit_smi in rdkit_smis:
        try:
            rdkit_mol = modifify_mol(rdkit_smi)
            Chem.SanitizeMol(rdkit_mol)
            rdkit_smi = Chem.MolToSmiles(rdkit_mol, isomericSmiles=True)
        except:
            continue
        mols.append(Chem.MolFromSmiles(rdkit_smi, sanitize=False))
    return mols

# Load Reaxys Data (Chlorination Dataset)

In [29]:
%cd {local_data}/raw_reaxys_data

df1 = pd.read_excel("Reaxys_Exp_20220506_1-5000.xlsx", )
df2 = pd.read_excel("Reaxys_Exp_20220506_5001-10000.xlsx", )
df3 = pd.read_excel("Reaxys_Exp_20220506_10001-15000.xlsx", )
df4 = pd.read_excel("Reaxys_Exp_20220506_15001-15476.xlsx", )

%cd {work_dir}


dataframes = [df1, df2, df3, df4]
df = pd.concat(dataframes, ignore_index = True)
print(f'Shape of DataFrame: {df.shape}')
print(f'Unique compounds in DataFrame: {len(set(list(df["Reaction ID"])))}')

/Users/hmm/PhD/Reaxys_Datasets/2022-05-06/raw_reaxys_data


/Users/hmm/opt/anaconda3/envs/my-rdkit-env/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/hmm/opt/anaconda3/envs/my-rdkit-env/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/hmm/opt/anaconda3/envs/my-rdkit-env/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/hmm/opt/anaconda3/envs/my-rdkit-env/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's de

/Users/hmm/PhD/JupyterNotebooks
Shape of DataFrame: (27734, 41)
Unique compounds in DataFrame: 15478


In [30]:
df.head(1)
len(df), len(set(df["Reaction ID"]))

(27734, 15478)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27734 entries, 0 to 27733
Data columns (total 41 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Reaction ID                                      27734 non-null  object 
 1   Reaction: Links to Reaxys                        27726 non-null  object 
 2   Data Count                                       27726 non-null  object 
 3   Number of Reaction Details                       27726 non-null  float64
 4   Reaction Rank                                    27726 non-null  float64
 5   Record Type                                      27726 non-null  object 
 6   Reactant                                         27726 non-null  object 
 7   Product                                          27726 non-null  object 
 8   Bin                                              24327 non-null  object 
 9   Reaction                    

# Exploratory Data Exploration

In [32]:
def FMCS_atoms(mol1, mol2):
    """ Find maximum common substructure.
        return: list of atoms indexes for mol2 """

    mol1_copy = copy.deepcopy(mol1)
    mol1_copy.UpdatePropertyCache(strict=False)
    rdmolops.SetAromaticity(mol1_copy)

    mol2_copy = copy.deepcopy(mol2)
    mol2_copy.UpdatePropertyCache(strict=False)
    rdmolops.SetAromaticity(mol2_copy)

    mols = [mol1_copy, mol2_copy]
    res = rdFMCS.FindMCS(mols, atomCompare=rdFMCS.AtomCompare.CompareAnyHeavyAtom, bondCompare=rdFMCS.BondCompare.CompareAny, ringMatchesRingOnly=True)

    return mol2_copy.GetSubstructMatch(Chem.MolFromSmarts(res.smartsString))

In [33]:
def get_site_selectivity(reac_mol, prod_mol):
    
    # Make a copy of the mols
    reac_mol_copy = copy.deepcopy(reac_mol)
    prod_mol_copy = copy.deepcopy(prod_mol)

    # Create single bonded mols
    single_bonded_reac = single_bonded_mol(reac_mol_copy)
    single_bonded_prod = single_bonded_mol(prod_mol_copy)

    # Compare reac_mol and prod_mol
    MCS_atoms = FMCS_atoms(reac_mol_copy, prod_mol_copy)
    atoms = single_bonded_prod.GetSubstructMatch(single_bonded_reac)

    if len(MCS_atoms) != reac_mol_copy.GetNumHeavyAtoms(): #check if the atoms in the MCS match the number of atoms in the reactant
        #print(len(MCS_atoms), reac_mol_copy.GetNumAtoms())
        raise ValueError('Reactant And Product Error I')
    elif len(MCS_atoms) == prod_mol_copy.GetNumHeavyAtoms(): #check if the product contains other atoms than just the MCS
        #print(len(MCS_atoms), prod_mol_copy.GetNumAtoms())
        raise ValueError('Reactant And Product Error II')

    for a in single_bonded_prod.GetAtoms():
        if a.GetIdx() not in MCS_atoms and a.GetSymbol() != 'Cl': #check if the atoms of the product not included in the MCS are chorine
            #print(a.GetIdx(), a.GetSymbol())
            raise ValueError('Reactant And Product Error III')


    # Find chlorine in product mol
    pat = Chem.MolFromSmarts("[#17]")
    chlorine_in_product = [atom for element in prod_mol_copy.GetSubstructMatches(pat) for atom in element] 
        
    if len(chlorine_in_product) == prod_mol_copy.GetNumAtoms(): #check if all atoms in products are chlorine
        #print(len(chlorine_in_product), prod_mol_copy.GetNumAtoms())
        raise ValueError('Reactant And Product Error IV')


    # Find chlorine in reactant mol
    pat = Chem.MolFromSmarts("[#17]")
    chlorine_in_reactant = [atom for element in reac_mol_copy.GetSubstructMatches(pat) for atom in element]
    
    if len(chlorine_in_reactant) == reac_mol_copy.GetNumAtoms(): #check if all atoms in reactant are chlorine
        #print(len(chlorine_in_reactant), reac_mol_copy.GetNumAtoms())
        raise ValueError('Reactant And Product Error V') 
    
    # Find reaction centers
    if len(chlorine_in_reactant):
        chlorine_in_reactant = [atoms[idx] for idx in chlorine_in_reactant]
    

    prod_mol_copy.UpdatePropertyCache(strict=False)
    rdmolops.SetAromaticity(prod_mol_copy)
    reaction_centers = []
    for atom in prod_mol_copy.GetAtoms():
        for atomNeighbor in atom.GetNeighbors():

            if atomNeighbor.GetSymbol() == 'Cl' and atomNeighbor.GetIdx() not in chlorine_in_reactant:
                
                if not atom.GetIsAromatic() or atom.GetAtomicNum() != 6: #check if the chlorine is attached to an aromatic carbon atom
                # if not atom.IsInRing() or atom.GetAtomicNum() != 6 or len(atom.GetBonds()) != 3: #check if the chlorine is attached to a cyclic SP2 hybridized carbon atom (THIS COULD ALSO BE USED IN THE FUTURE)
                    raise ValueError('Reactant And Product Error VI')
                else:
                    reaction_centers.append(atom.GetIdx())
    
    # Make sure that the reaction centers correspond to the atomnumbering of the reactant
    reaction_centers = reorder_labels(reac_mol, prod_mol, reaction_centers)

    return reaction_centers

In [34]:
%%time
reactants_dataset2 = []
reaction_centers_dataset2 = []
reaction_id2 = []

for k, reaction in tqdm(df.groupby(['Reaction ID'])):
    reactants = []
    reaction_centers = []
    for idx, exp_obs in reaction.iterrows():
        rxn_smiles = exp_obs['Reaction']
        if rxn_smiles is np.nan:
           continue

        rxn_reactants = rxn_smiles.split('>>')[0].split('.')
        rxn_products = rxn_smiles.split('>>')[1].split('.')

        reac_mols, reac_smiles = get_unique_reps2(rxn_reactants)
        prod_mols, prod_smiles = get_unique_reps2(rxn_products)

        for reac_mol, reac_smi in zip(reac_mols, reac_smiles):
            reaction_okay = False
            for prod_mol, prod_smi in zip(prod_mols, prod_smiles):
                
                if reac_smi == prod_smi:
                    continue
                
                try:
                    rc = get_site_selectivity(reac_mol, prod_mol)
                except:
                    continue
                
                reaction_centers.extend(rc)
                reaction_okay = True
                
            if reaction_okay and reac_smi not in reactants:
                reactants.append(reac_smi)
        reaction_centers = np.unique(reaction_centers).tolist()

    if len(reactants) == 1 and len(reaction_centers):
        reactants_dataset2.append(reactants)
        reaction_centers_dataset2.append(reaction_centers)
        reaction_id2.append(k)

 22%|██▏       | 3402/15478 [01:58<08:16, 24.31it/s]  [10:41:24] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:41:24] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:41:24] SMILES Parse Error: unclosed ring for input: 'ClC1=CC=C(C=C1)[Pt]123[Cl][Pt]4([Cl]1)(C1=CC=C(Cl)C=C1)[CH]1=[CH]4CCC(=C)CCC1'
[10:41:24] SMILES Parse Error: unclosed ring for input: 'C=C1CCC[CH]2=[CH]3CC1'
[10:41:24] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:41:24] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:41:24] SMILES Parse Error: unclosed ring for input: 'ClC1=CC=C(C=C1)[Pt]123[Cl][Pt]4([Cl]1)(C1=CC=C(Cl)C=C1)[CH]1=[CH]4CCC(=C)CCC1'
[10:41:24] SMILES Parse Error: unclosed ring for input: 'C=C1CCC[CH]2=[CH]3CC1'
 22%|██▏       | 3407/15478 [01:58<07:40, 26.21it/s][10:41:24] Explicit valence for atom # 1 C, 5, is greater than permitted
[10:41:24] Explicit valence for atom # 1 C, 5, is greater than permitted
[10:41:24] Explicit vale

CPU times: user 17min 6s, sys: 25 s, total: 17min 31s
Wall time: 19min 5s


In [35]:
df['Reaction ID'].isin(reaction_id2)

0        False
1        False
2        False
3        False
4        False
         ...  
27729    False
27730     True
27731     True
27732    False
27733    False
Name: Reaction ID, Length: 27734, dtype: bool

In [36]:
new_df = df[df['Reaction ID'].isin(reaction_id2)]
new_df.sort_values(by="Yield", ascending=False, inplace=True, key=lambda x: natsorted(new_df["Yield"]))
new_df = new_df.groupby(['Reaction ID']).head(1)
new_df.reset_index(drop=True, inplace=True)
new_df.shape

/var/folders/jp/w62zl0z932s8hz_k3b5m5wk80000gn/T/ipykernel_79995/656130862.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.sort_values(by="Yield", ascending=False, inplace=True, key=lambda x: natsorted(new_df["Yield"]))


(5504, 41)

In [37]:
new_df['Reactant Smiles'] = None
new_df['Reaction Centers'] = None
for index, row in new_df.iterrows():
    new_df.at[index, 'Reactant Smiles'] = reactants_dataset2[reaction_id2.index(row['Reaction ID'])][0]
    new_df.at[index, 'Reaction Centers'] = reaction_centers_dataset2[reaction_id2.index(row['Reaction ID'])]
new_df.shape

/var/folders/jp/w62zl0z932s8hz_k3b5m5wk80000gn/T/ipykernel_79995/2616834694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Reactant Smiles'] = None
/var/folders/jp/w62zl0z932s8hz_k3b5m5wk80000gn/T/ipykernel_79995/2616834694.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Reaction Centers'] = None


(5504, 43)

In [38]:
print(f'Unique reactions in DataFrame: {len(set(list(df["Reaction ID"])))}')
print('Compounds left:', len(reactants_dataset2), len(reaction_centers_dataset2))
print('Unique reactants:', len(set(reduce(operator.concat, reactants_dataset2))))

Unique reactions in DataFrame: 15478
Compounds left: 5504 5504
Unique reactants: 4854


In [39]:
!pwd

/Users/hmm/PhD/JupyterNotebooks


In [41]:
# Write data set to file - with some cleaning for reactant and product EAS sites locations

%cd {local_data}

f = open('./reaxys_chlorination_dataset2.txt','w')
g = open('./reaxys_chlorination_conflicts.txt', "w")

for index, reaction in tqdm(new_df.groupby(['Reactant Smiles'])): 
    if len(reaction) > 1:
        reaction_id = []
        reaction_smiles = []
        reaction_centers = []
        for reac_id, reac_smi, rc in zip(reaction['Reaction ID'], reaction['Reactant Smiles'], reaction['Reaction Centers']):
            write_okay = True
            measured_atoms = rc
            measured_atoms = find_identical_atoms(reac_smi, measured_atoms)
            rdkit_mol = Chem.MolFromSmiles(reac_smi)
            eas_sites = sorted(locate_sites(rdkit_mol))
            for matom in measured_atoms:
                if matom not in eas_sites:
                    print(reac_id, reac_smi, measured_atoms, eas_sites) # reaction does not obey the reaction rules
                    write_okay = False
                    # raise ValueError('Measured site not in located EAS sites')
                    break
            if not write_okay:
                continue
            reaction_id.append(str(reac_id))
            reaction_smiles.append(reac_smi)
            reaction_centers.extend(rc)
        
        if len(reaction_smiles):
            write_okay = True
            reaction_id = '_'.join(reaction_id)
            reaction_smiles = reaction_smiles[0]
            reaction_centers = ','.join(map(str, np.unique(reaction_centers)))
    else:
        reaction_id = reaction['Reaction ID'].values[0]
        reaction_smiles = reaction['Reactant Smiles'].values[0]
        reaction_centers = reaction['Reaction Centers'].values[0]

        write_okay = True
        measured_atoms = reaction_centers
        measured_atoms = find_identical_atoms(reaction_smiles, measured_atoms)
        rdkit_mol = Chem.MolFromSmiles(reaction_smiles)
        eas_sites = sorted(locate_sites(rdkit_mol))
        for matom in measured_atoms:
            if matom not in eas_sites:
                print(reaction_id, reaction_smiles, measured_atoms, eas_sites) # reaction does not obey the reaction rules
                write_okay = False
                # raise ValueError('Measured site not in located EAS sites')
                break

        reaction_centers = ','.join(map(str, np.unique(reaction_centers)))
    
    if write_okay:
        f.write(f"{reaction_id}    {reaction_smiles}    {reaction_centers}\n")
    else:
        g.write(f"{reaction_id}    {reaction_smiles}    {measured_atoms}    {eas_sites}\n")
f.close()
%cd {work_dir}

/Users/hmm/PhD/Reaxys_Datasets/2022-05-06


  3%|▎         | 139/4854 [00:01<00:22, 212.28it/s]

39349381 C=CCOC1=CC(=O)CCC1 [8] [5]
39349380 C=CCOC1=CC(=O)CCC1 [10] [5]


  7%|▋         | 320/4854 [00:01<00:18, 251.70it/s]

39349363 CC(C)(C)C(=O)OC1=CC(=O)CCC1 [11] [8]


...

2216302 Ic1cccc2c1NCC2 [9] [2, 3, 4]


 87%|████████▋ | 4247/4854 [00:14<00:02, 260.14it/s]

39349357 O=C1C=C(N2CCCC2)CCC1 [9] [2]
39349383 O=C1C=C(OCc2ccccc2)CCC1 [12] [2, 7, 8, 9, 10, 11]
39349392 O=C1C=C(Oc2ccccc2)CCC1 [11] [2, 6, 7, 8, 9, 10]
39349355 O=C1C=C(Sc2ccccc2)CCC1 [13] [2, 6, 7, 8, 9, 10]
53678217 O=C1CCCCN1 [2] []
51616666 O=CC1Cc2ccccc2S1 [3] [5, 6, 7, 8]


100%|██████████| 4854/4854 [00:17<00:00, 281.09it/s]

/Users/hmm/PhD/JupyterNotebooks


In [28]:
def read_input(smiles_filename, exam=True):

    f = open(smiles_filename, "r")

    names_list = []
    input_smiles_list = []
    atom_measured_list = []
    for line in f:
        words = line.split()
        names_list.append(words[0])
        input_smiles_list.append(words[1])

        if exam:
            atom_measured = [int(x) for x in words[2].split(",")]
            atom_measured = find_identical_atoms(words[1], atom_measured)
            atom_measured_list.append(atom_measured)
        else:
            atom_measured_list.append(None)
    f.close()

    return names_list, input_smiles_list, atom_measured_list

In [29]:
%cd {local_data}
names_list, input_smiles_list, atom_measured_list = read_input('reaxys_chlorination_dataset2.txt')
%cd {work_dir}

/Users/hmm/PhD/Reaxys_Datasets/2022-05-06
/Users/hmm/PhD/JupyterNotebooks


In [30]:
len(names_list)

4824

# Looking for conflicting reactions

Definition: a conflicting reaction pair have the same starting material but give different chlorination products. 

An encountered issue observed are reactions that have entries with multiple reaction sites and others that have a subset of those. This might arise form shorter reaction times, or lesser equivalents of chlorination agents.

In [31]:
reactants_dataset2 = []
reaction_centers_dataset2 = []
reaction_id2 = []
conflicting = []
total_df = pd.DataFrame(columns = ("ID", "Reactant", "Rxt_Site", "Conflict"))
total_df

,ID,Reactant,Rxt_Site,Conflict


In [32]:
# Iterate over the reaction list and create an output that gives the following:
# For each reaction, the rxt ID + reactant + reaction site + conflicting (T/F)

reactants_dataset2 = []
reaction_centers_dataset2 = []
reaction_id2 = []
conflicting = []

for k, reaction in tqdm(df.groupby(['Reaction ID'])):
    reactants = []
    reaction_centers = []
    for idx, exp_obs in reaction.iterrows():
        rxn_smiles = exp_obs['Reaction']
        if rxn_smiles is np.nan:
           continue

        rxn_reactants = rxn_smiles.split('>>')[0].split('.')
        rxn_products = rxn_smiles.split('>>')[1].split('.')

        reac_mols, reac_smiles = get_reps2(rxn_reactants)
        prod_mols, prod_smiles = get_reps2(rxn_products)

        for reac_mol, reac_smi in zip(reac_mols, reac_smiles):
            reaction_okay = False
            for prod_mol, prod_smi in zip(prod_mols, prod_smiles):
                
                if reac_smi == prod_smi:
                    continue
                
                try:
                    rc = get_site_selectivity(reac_mol, prod_mol)
                except:
                    continue
                
                reaction_centers.extend(rc)
                reaction_okay = True
                
            if reaction_okay and reac_smi not in reactants:
                reactants.append(reac_smi)
        reaction_centers = np.unique(reaction_centers).tolist()

    if len(reactants) == 1 and len(reaction_centers):
        reactants_dataset2.append(reactants[0])
        reaction_centers_dataset2.append(reaction_centers)
        reaction_id2.append(k)
        conflicting.append(False)

 22%|██▏       | 3404/15478 [02:01<10:35, 18.99it/s]  [15:30:09] Explicit valence for atom # 3 C, 5, is greater than permitted
[15:30:09] Explicit valence for atom # 3 C, 5, is greater than permitted
[15:30:09] SMILES Parse Error: unclosed ring for input: 'ClC1=CC=C(C=C1)[Pt]123[Cl][Pt]4([Cl]1)(C1=CC=C(Cl)C=C1)[CH]1=[CH]4CCC(=C)CCC1'
[15:30:09] SMILES Parse Error: unclosed ring for input: 'C=C1CCC[CH]2=[CH]3CC1'
[15:30:09] Explicit valence for atom # 3 C, 5, is greater than permitted
[15:30:09] Explicit valence for atom # 3 C, 5, is greater than permitted
[15:30:09] SMILES Parse Error: unclosed ring for input: 'ClC1=CC=C(C=C1)[Pt]123[Cl][Pt]4([Cl]1)(C1=CC=C(Cl)C=C1)[CH]1=[CH]4CCC(=C)CCC1'
[15:30:09] SMILES Parse Error: unclosed ring for input: 'C=C1CCC[CH]2=[CH]3CC1'
[15:30:09] Explicit valence for atom # 1 C, 5, is greater than permitted
[15:30:09] Explicit valence for atom # 1 C, 5, is greater than permitted
[15:30:09] Explicit valence for atom # 1 C, 5, is greater than permitted
[15

In [33]:
len(reactants_dataset2) , len(reaction_centers_dataset2), len(reaction_id2), len(conflicting)

(5504, 5504, 5504, 5504)

In [34]:
total_df = pd.DataFrame(columns = ("ID", "Reactant", "Rxt_Site", "Conflict"))
total_df["ID"] = reaction_id2
total_df["Reactant"] = reactants_dataset2
total_df["Rxt_Site"] = reaction_centers_dataset2
total_df["Conflict"] = conflicting
total_df

,ID,Reactant,Rxt_Site,Conflict
0,10002925,CC(C)(C)OC(=O)c1cn(-c2ccc(Cl)cc2)c(-c2ccc(Cl)c...,[8],False
1,10023067,COc1cccc(N)n1,"[3, 5]",False
2,10026626,Fc1cccnc1F,[2],False
3,10027100,Fc1ncccc1Cl,[5],False
4,10027717,Fc1ccccn1,[2],False
...,...,...,...,...
5499,9993498,CCN(CC)C(=O)Oc1cccc(I)c1,[14],False
5500,9993501,CCN(CC)C(=O)Oc1cccc(Br)c1,[14],False
5501,9993991,CC(C)(C)OC(=O)Oc1ccc2cc[nH]c2c1,[12],False
5502,9997597,CCCCCC(=O)c1c(O)cc(OC)cc1O,"[10, 14]",False


In [35]:
## Get the previous list and for each entry look for:
    # If another entry with same reactant && conflicting == False
        ## If reaction sites are different, conflicting = True
        
for react_iter in range(len(total_df["Reactant"])):
    for compare_iter in range(len(total_df["Reactant"])):
        react = total_df["Reactant"][react_iter]
        compare = total_df["Reactant"][compare_iter]
    # If another entry with same reactant AND conflicting == False
        if react == compare and react_iter != compare_iter and total_df["Conflict"][react_iter] == False:
            rxt_site = total_df["Rxt_Site"][react_iter]
            compare_site = total_df["Rxt_Site"][compare_iter]
            
            if rxt_site != compare_site:
                total_df["Conflict"][react_iter] = True
                
sum(total_df["Conflict"])

/var/folders/jp/w62zl0z932s8hz_k3b5m5wk80000gn/T/ipykernel_55393/4124123186.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df["Conflict"][react_iter] = True


900

In [36]:
conflicting_df = total_df.copy()
conflicting_df = conflicting_df[conflicting_df.Conflict == True]
conflicting_df = conflicting_df.sort_values(by = ["Reactant"])

In [37]:
conflicting_df.index = pd.RangeIndex(len(conflicting_df.index))
conflicting_df

,ID,Reactant,Rxt_Site,Conflict
0,39565104,Brc1ccc(-c2ccccn2)cc1,[3],True
1,37125432,Brc1ccc(-c2ccccn2)cc1,"[3, 11]",True
2,29236441,C=C(Cl)CNc1ccc([N+](=O)[O-])cc1,[13],True
3,29236442,C=C(Cl)CNc1ccc([N+](=O)[O-])cc1,"[6, 13]",True
4,39349381,C=CCOC1=CC(=O)CCC1,[8],True
...,...,...,...,...
895,2189389,c1csc(-c2cccs2)c1,"[1, 5, 7, 9]",True
896,4726774,c1csc(-c2cccs2)c1,"[1, 6, 7, 9]",True
897,4726773,c1csc(-c2cccs2)c1,"[1, 5, 7]",True
898,47634166,c1cscn1,"[1, 3]",True


In [38]:
len(set(list(conflicting_df["Reactant"])))

361